[![Renode](https://dl.antmicro.com/projects/renode/renode.svg)](https://renode.io)

[![Run in Google Colab](https://img.shields.io/badge/-Run%20in%20Google%20colab-%23007ded?logo=google-colab&logoColor=white&style=for-the-badge)](https://colab.research.google.com/github/antmicro/test-colabs/blob/main/boards/st25dv_mb1283_disco_lz4.ipynb) [![View ipynb](https://img.shields.io/badge/-View%20ipynb%20source-%23007ded?logo=jupyter&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/st25dv_mb1283_disco_lz4.ipynb) [![View Python source](https://img.shields.io/badge/-View%20Python%20source-%23007ded?logo=python&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/st25dv_mb1283_disco_lz4.py)

## Install requirements

In [ ]:
! pip install -q git+https://github.com/antmicro/renode-colab-tools.git
! pip install -q git+https://github.com/antmicro/renode-run.git
! pip install -q git+https://github.com/antmicro/pyrenode.git
! renode-run download

## Start Renode

In [ ]:
from pyrenode import connect_renode, get_keywords
connect_renode()
get_keywords()

## Setup a script

In [ ]:
%%writefile script.resc

using sysbus
$name?="st25dv_mb1283_disco"
mach create $name

machine LoadPlatformDescription @https://zephyr-dashboard.renode.io/zephyr_sim/a70bf0e488c5327ad67512c2854987d113318c79/39825dfac1127ddb969d5fea99613f72ad37c639/st25dv_mb1283_disco/lz4/lz4.repl
machine EnableProfiler $ORIGIN/metrics.dump

showAnalyzer sysbus.usart6
sysbus.usart6 RecordToAsciinema $ORIGIN/output.asciinema

macro reset
"""
    sysbus LoadELF @https://zephyr-dashboard.renode.io/zephyr/a70bf0e488c5327ad67512c2854987d113318c79/st25dv_mb1283_disco/lz4/lz4.elf
    
"""

runMacro $reset

## Run the sample

In [ ]:
ExecuteScript("script.resc")
CreateTerminalTester("sysbus.usart6", timeout=5)
StartEmulation()

WaitForLineOnUart(r"Original Data size: \d+", treatAsRegex=True)
WaitForLineOnUart(r"Compressed Data size : \d+", treatAsRegex=True)
WaitForLineOnUart("Successfully decompressed some data")
WaitForLineOnUart("Validation done. The string we ended up with is:")
WaitForLineOnUart(r".*", treatAsRegex=True)

ResetEmulation()

## UART output

In [ ]:
from renode_colab_tools import asciinema
asciinema.display_asciicast('output.asciinema')

## Renode metrics analysis

In [ ]:
import sys
from pathlib import Path
from renode_run import get_default_renode_path
sys.path.append(str(Path(get_default_renode_path()).parent))

from renode_colab_tools import metrics
from tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('metrics.dump')

metrics.display_metrics(parser)